In [ ]:
import pandas as pd
import numpy as np
import datetime
import copy

In [ ]:
#U用户在一个时间窗口内的浏览(1)、收藏(2)、加购物车(3)、购买(4)的行为次数，总行为次数。
#user_behavior_data: 时间窗口内的数据
def get_feature_u(user_behavior_data):
    user_behavior_data['cumcount'] = user_behavior_data.groupby(['user_id','behavior_type']).cumcount()
    #print('user_behavior_data')
    #print(user_behavior_data.head(5))
    user_behavior_data_u = user_behavior_data.drop_duplicates(['user_id','behavior_type'], 'last')[['user_id','behavior_type','cumcount']]
    #print('user_behavior_data_u-1')
    #print(user_behavior_data_u.head(5))
    user_behavior_data_u = pd.get_dummies(user_behavior_data_u['behavior_type']).join(user_behavior_data_u[['user_id','cumcount']])
    #print('user_behavior_data_u-2')
    #print(user_behavior_data_u.head(5))
    user_behavior_data_u.rename(columns = {1:'behavior_type_1',
                                                               2:'behavior_type_2',
                                                               3:'behavior_type_3',
                                                               4:'behavior_type_4'}, inplace=True)
    #print('user_behavior_data_u-3')
    #print(user_behavior_data_u.head(5))
    user_behavior_data_u['u_b1_count'] = user_behavior_data_u['behavior_type_1'] * (user_behavior_data_u['cumcount']+1)
    user_behavior_data_u['u_b2_count'] = user_behavior_data_u['behavior_type_2'] * (user_behavior_data_u['cumcount']+1)
    user_behavior_data_u['u_b3_count'] = user_behavior_data_u['behavior_type_3'] * (user_behavior_data_u['cumcount']+1)
    user_behavior_data_u['u_b4_count'] = user_behavior_data_u['behavior_type_4'] * (user_behavior_data_u['cumcount']+1)
    #print('user_behavior_data_u-4')
    #print(user_behavior_data_u.head(5))
    #这步没做什么，实质是进行了列的选取
    user_behavior_data_u = user_behavior_data_u.groupby('user_id').agg({'u_b1_count': np.sum,
                                                                                                      'u_b2_count': np.sum, 
                                                                                                      'u_b3_count': np.sum, 
                                                                                                      'u_b4_count': np.sum})
    #print('user_behavior_data_u-5')
    #print(user_behavior_data_u.head(5))

    user_behavior_data_u.reset_index(inplace = True)
    user_behavior_data_u['u_b_count'] = user_behavior_data_u[['u_b1_count',
                                                                                        'u_b2_count',
                                                                                        'u_b3_count',
                                                                                        'u_b4_count']].apply(lambda x: x.sum(), axis = 1)
    #print('user_behavior_data_u-6')
    #print(user_behavior_data_u.head(5))
    return user_behavior_data_u

#I商品在目标日前6天的各类行为总数、总行为次数、商品的点击购买转化率、商品的点击购买平均时差、被访问人数。
def get_feature_i(user_behavior_data):
    user_behavior_data_copy = user_behavior_data
    user_behavior_data_copy['i_cumcount'] = user_behavior_data_copy.groupby(['item_id','behavior_type']).cumcount()
    user_behavior_data_i = user_behavior_data_copy.drop_duplicates(['item_id','behavior_type'], 'last')[['item_id','behavior_type','i_cumcount']]
    user_behavior_data_i = pd.get_dummies(user_behavior_data_i['behavior_type']).join(user_behavior_data_i[['item_id','i_cumcount']])
    user_behavior_data_i.rename(columns = {1:'i_behavior_type_1',
                                           2:'i_behavior_type_2',
                                           3:'i_behavior_type_3',
                                           4:'i_behavior_type_4'}, inplace=True)
    ##各类行为次数
    user_behavior_data_i['i_b1_count'] = user_behavior_data_i['i_behavior_type_1'] * (user_behavior_data_i['i_cumcount']+1)
    user_behavior_data_i['i_b2_count'] = user_behavior_data_i['i_behavior_type_2'] * (user_behavior_data_i['i_cumcount']+1)
    user_behavior_data_i['i_b3_count'] = user_behavior_data_i['i_behavior_type_3'] * (user_behavior_data_i['i_cumcount']+1)
    user_behavior_data_i['i_b4_count'] = user_behavior_data_i['i_behavior_type_4'] * (user_behavior_data_i['i_cumcount']+1)
    ##这步其实没做什么，实质是进行了列的选取
    user_behavior_data_i = user_behavior_data_i.groupby('item_id').agg({'i_b1_count': np.sum,
                                                                            'i_b2_count': np.sum, 
                                                                            'i_b3_count': np.sum, 
                                                                            'i_b4_count': np.sum})
    user_behavior_data_i.reset_index(inplace = True)
    ##总行为次数
    user_behavior_data_i['i_b_count'] = user_behavior_data_i[['i_b1_count',
                                                                                     'i_b2_count',
                                                                                     'i_b3_count',
                                                                                     'i_b4_count']].apply(lambda x: x.sum(), axis = 1)
    ##商品点击购买转化率
    user_behavior_data_i['i_b4_rate'] = user_behavior_data_i['i_b4_count'] / user_behavior_data_i['i_b_count']
    ##商品点击购买平均时差（非常重要）；不用的人对同一个商品的时差不一致，需要平均。另外这个购买平均时差的特征加持在人身上效果应该会更好，
    #暂略

    ##被访问人数
    item_user_group_size = pd.DataFrame(user_behavior_data.groupby(['item_id','user_id']).size())
    item_user_num = pd.DataFrame(item_user_group_size.groupby(['item_id']).size())
    item_user_num.rename(columns = {0:'i_user_num'},inplace=True)
    user_behavior_data_i=user_behavior_data_i.set_index('item_id')
    user_behavior_data_i = user_behavior_data_i.join(item_user_num)
    user_behavior_data_i.reset_index(inplace = True)
    return user_behavior_data_i

#C类目在目标日前6天的各类行为总数、总行为次数、商品的点击购买转化率、商品的点击购买平均时差、被访问人数。
def get_feature_c(user_behavior_data):
    user_behavior_data_copy = user_behavior_data
    #df_part_1_c_b = df_part_1_u_b
    user_behavior_data_copy['c_cumcount'] = user_behavior_data_copy.groupby(['item_category','behavior_type']).cumcount()
    user_behavior_data_c = user_behavior_data_copy.drop_duplicates(['item_category','behavior_type'], 'last')[['item_category','behavior_type','c_cumcount']]
    user_behavior_data_c = pd.get_dummies(user_behavior_data_c['behavior_type']).join(user_behavior_data_c[['item_category','c_cumcount']])
    user_behavior_data_c.rename(columns = {1:'c_behavior_type_1',
                                           2:'c_behavior_type_2',
                                           3:'c_behavior_type_3',
                                           4:'c_behavior_type_4'}, inplace=True)
    ##各类行为次数
    user_behavior_data_c['c_b1_count'] = user_behavior_data_c['c_behavior_type_1'] * (user_behavior_data_c['c_cumcount']+1)
    user_behavior_data_c['c_b2_count'] = user_behavior_data_c['c_behavior_type_2'] * (user_behavior_data_c['c_cumcount']+1)
    user_behavior_data_c['c_b3_count'] = user_behavior_data_c['c_behavior_type_3'] * (user_behavior_data_c['c_cumcount']+1)
    user_behavior_data_c['c_b4_count'] = user_behavior_data_c['c_behavior_type_4'] * (user_behavior_data_c['c_cumcount']+1)
    ##这步其实没做什么，实质是进行了列的选取
    user_behavior_data_c = user_behavior_data_c.groupby('item_category').agg({'c_b1_count': np.sum,
                                                                                  'c_b2_count': np.sum, 
                                                                                  'c_b3_count': np.sum, 
                                                                                  'c_b4_count': np.sum})
    user_behavior_data_c.reset_index(inplace = True)
    ##总行为次数
    user_behavior_data_c['c_b_count'] = user_behavior_data_c[['c_b1_count',
                                                                       'c_b2_count',
                                                                       'c_b3_count',
                                                                       'c_b4_count']].apply(lambda x: x.sum(), axis = 1)
    ##商品点击购买转化率
    user_behavior_data_c['c_b4_rate'] = user_behavior_data_c['c_b4_count'] / user_behavior_data_c['c_b_count']

    ##商品点击购买平均时差（非常重要）；不用的人对同一个商品的时差不一致，需要平均。另外这个购买平均时差的特征加持在人身上效果应该会更好，
    #暂略

    ##被访问人数
    category_user_group_size = pd.DataFrame(user_behavior_data_copy.groupby(['item_category','user_id']).size())
    category_user_num = pd.DataFrame(category_user_group_size.groupby(['item_category']).size())
    category_user_num.rename(columns = {0:'c_user_num'},inplace=True)
    user_behavior_data_c=user_behavior_data_c.set_index(['item_category'])
    user_behavior_data_c = user_behavior_data_c.join(category_user_num)
    user_behavior_data_c.reset_index(inplace = True)
    return user_behavior_data_c

#IC商品在所属类别中的用户人数(排序)、商品在所属类别中的行为总数(排序)、商品在所属类别中的销量排序
def get_feature_ic(user_behavior_data): 
    #df_part_1_ic_b = df_part_1_u_b
    user_behavior_data_copy = user_behavior_data
    ##商品在所属类别中的用户人数
    item_category_user_size = pd.DataFrame(user_behavior_data_copy.groupby(['item_id','item_category','user_id']).size())
    item_category_user_num = pd.DataFrame(item_category_user_size.groupby(['item_id','item_category']).size())
    item_category_user_num.rename(columns = {0:'ic_user_num'},inplace=True)
    #print(item_category_user_num.head(5))
    ##商品在所属类别中的行为总数
    item_category_behavior_size = pd.DataFrame(user_behavior_data_copy.groupby(['item_id','item_category']).size())
    item_category_behavior_size.rename(columns = {0:'ic_behavior_num'},inplace=True)
    #print('1:',item_category_behavior_size.head(5))
    ##商品在所属类别中的销量排序
    item_category_behavior_sales_size = pd.DataFrame(user_behavior_data_copy.groupby(['item_id','item_category','behavior_type']).size())
    item_category_behavior_sales_size = item_category_behavior_sales_size.reset_index()
    #print('2:',item_category_behavior_sales_size.head(5))
    user_behavior_data_ic = item_category_behavior_sales_size[item_category_behavior_sales_size.behavior_type==4]
    user_behavior_data_ic.rename(columns = {0:'ic_sales_num'},inplace=True)
    user_behavior_data_ic.set_index(['item_id','item_category'],inplace=True)
    #print('3:',df_part_1_ic_b_count_in_6.head(5))
    user_behavior_data_ic = user_behavior_data_ic.join(item_category_user_num,how='outer')
    user_behavior_data_ic = user_behavior_data_ic.join(item_category_behavior_size,how='outer')
    user_behavior_data_ic = user_behavior_data_ic[['ic_sales_num','ic_user_num','ic_behavior_num']].fillna(0)
    #df_part_1_ic_b_count_in_6
    user_behavior_data_ic.reset_index(inplace = True)
    return user_behavior_data_ic

    
#UI用户-商品对在考察日前6天的行为总数计数、用户-商品对在考察日前6天的各项行为计数、
#用户-商品对各项行为上一次发生距考察日的时差、用户商品对的行为在用户所有商品中的排序
def get_feature_ui(user_behavior_data):  
    #df_part_1_ui_b = df_part_1_u_b
    user_behavior_data_copy = user_behavior_data
    
    ui_b_count_size = pd.DataFrame(user_behavior_data_copy.groupby(['item_id','user_id','behavior_type']).size())
    ui_b_count_size = ui_b_count_size.reset_index()
    ui_b_count_size.rename(columns={0:'ui_cumcount'},inplace=True)
    ui_b_count_num = pd.get_dummies(ui_b_count_size['behavior_type']).join(ui_b_count_size[['user_id','item_id','ui_cumcount']])
    ui_b_count_num.rename(columns={1:'ui_b1_count',
                                        2:'ui_b2_count',
                                        3:'ui_b3_count',
                                        4:'ui_b4_count'},inplace=True)
    ##用户-商品对在考察日前n天的各项行为计数
    ui_b_count_num['ui_b1_count'] = ui_b_count_num['ui_b1_count'] * ui_b_count_num['ui_cumcount']
    ui_b_count_num['ui_b2_count'] = ui_b_count_num['ui_b2_count'] * ui_b_count_num['ui_cumcount']
    ui_b_count_num['ui_b3_count'] = ui_b_count_num['ui_b3_count'] * ui_b_count_num['ui_cumcount']
    ui_b_count_num['ui_b4_count'] = ui_b_count_num['ui_b4_count'] * ui_b_count_num['ui_cumcount']
    ##用户-商品对在考察日前n天的行为总数计数
    ui_b_count_num['ui_b_count']= ui_b_count_num['ui_b1_count'] + ui_b_count_num['ui_b2_count'] + ui_b_count_num['ui_b3_count'] + ui_b_count_num['ui_b4_count']
    ui_b_count_num = ui_b_count_num.groupby(['item_id','user_id']).sum()
    ##用户-商品对各项行为上一次发生距考察日的时差
    ##用户-商品对的行为在本用户所有用户-商品对中的排序
    ui_b_count_num = ui_b_count_num.sort_values(by='ui_b_count',ascending=True)
    temp = pd.DataFrame(ui_b_count_num.groupby(['user_id']).cumcount())
    ui_b_count_num = temp.join(ui_b_count_num)
    ui_b_count_num.rename(columns={0:'ui_sort'},inplace=True)
    ui_b_count_num['ui_sort'] = ui_b_count_num['ui_sort']+1
    ui_b_count_num.reset_index(inplace = True)
    return ui_b_count_num

#
def get_feature_uc(user_behavior_data):  
    user_behavior_data['cumcount'] = user_behavior_data.groupby(['user_id', 'item_category', 'behavior_type']).cumcount()
    uc_b_count = user_behavior_data.drop_duplicates(['user_id','item_category','behavior_type'],'last')[['user_id','item_category','behavior_type','cumcount']]
    uc_b_count = pd.get_dummies(uc_b_count['behavior_type']).join(uc_b_count[['user_id','item_category','cumcount']])
    uc_b_count.rename(columns = {1:'behavior_type_1',
                                                2:'behavior_type_2',
                                                3:'behavior_type_3',
                                                4:'behavior_type_4'}, inplace=True)  
    uc_b_count['uc_b1_count'] = uc_b_count['behavior_type_1'] * (uc_b_count['cumcount']+1)
    uc_b_count['uc_b2_count'] = uc_b_count['behavior_type_2'] * (uc_b_count['cumcount']+1)
    uc_b_count['uc_b3_count'] = uc_b_count['behavior_type_3'] * (uc_b_count['cumcount']+1)
    uc_b_count['uc_b4_count'] = uc_b_count['behavior_type_4'] * (uc_b_count['cumcount']+1)
    
    uc_b_count = uc_b_count[['user_id', 'item_category', 'uc_b1_count_in_6', 'uc_b2_count_in_6', 'uc_b3_count_in_6', 'uc_b4_count_in_6']]
    
    uc_b_count = uc_b_count.groupby(['user_id', 'item_category']).agg({'uc_b1_count': np.sum,
                                                                                                 'uc_b2_count': np.sum,
                                                                                                 'uc_b3_count': np.sum,
                                                                                                 'uc_b4_count': np.sum})
    uc_b_count.reset_index(inplace = True)
    uc_b_count['uc_b_count'] = uc_b_count['uc_b1_count'] + uc_b_count['uc_b2_count'] + uc_b_count['uc_b3_count'] + uc_b_count['uc_b4_count']
    pass

In [ ]:
def feature_construct(start_day, end_day, feature_window_size):
    start_time = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    #build file name
    second_end_day = (end_day - datetime.timedelta(days=1))    
    start_day_str = str(start_day.date())
    end_day_str = str(end_day.date())
    second_end_day_str = str(second_end_day.date())
    print('feature construct from %s  to %s start time:'%(start_day_str,end_day_str), start_time)
    name = start_day_str + "to" + second_end_day_str
    part_data_file_name = "../DataSet/"+"part_data"+name+".csv"
    #part_data_label_file_name =  "../DataSet/"+"part_data_label"+end_day_str+".csv"
    part_data_label_filename =  "../DataSet/"+"part_data_uicl"+end_day_str+".csv"
    user_behavior_data = pd.read_csv(part_data_file_name)
    user_behavior_data.columns = ['time','user_id','item_id','behavior_type','item_category']
    #output file name
    train_data_filename = "../DataSet/"+"part_data_uiclf"+end_day_str+".csv"
    #print(user_behavior_data.head(5))
    user_behavior_data_uic_label = pd.read_csv(part_data_label_filename)
    u=list()
    i_i=list()
    c=list()
    ic=list()
    ui=list()
    uc=list()
    for i in list([6,3,1]): 
        j=0  
        start_day = (end_day - datetime.timedelta(days=i))   
        end_day = end_day
        user_behavior_data = user_behavior_data[user_behavior_data['time'] >= start_day_str]
        
        #提取特征部分
        u.append(get_feature_u(copy.deepcopy(user_behavior_data)))
        i_i.append(get_feature_i(copy.deepcopy(user_behavior_data)))
        c.append(get_feature_c(copy.deepcopy(user_behavior_data)))
        ic.append(get_feature_ic(copy.deepcopy(user_behavior_data)))
        ui.append(get_feature_ui(copy.deepcopy(user_behavior_data)))
        uc.append(get_feature_uc(copy.deepcopy(user_behavior_data)))
        j+=1
        
    feature_construct_time = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    print('feature construct  time:', feature_construct_time)
    for i in list(range(len(u)))[1:]:
        u[0] = pd.merge(u[0], u[i], on = ['user_id'], how = 'left').fillna(0)
        i_i[0] = pd.merge(i_i[0], i_i[i], on = ['item_id'], how = 'left').fillna(0)
        c[0] = pd.merge(c[0], c[i], on = ['item_category'], how = 'left').fillna(0)
        ic[0] = pd.merge(ic[0], ic[i], on = ['item_id','item_category'], how = 'left').fillna(0)
        ui[0] = pd.merge(ui[0], ui[i], on = ['user_id','item_id'], how = 'left').fillna(0)
        #uc[0]= pd.merge(uc[0], uc[i], on = ['user_id','item_category'], how = 'left').fillna(0)
        
    feature_construct_join_time = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    print('feature construct join time:', feature_construct_join_time)
    #这个位置填充的合适的值是什么？   
    feature_data = pd.merge(user_behavior_data_uic_label, u[0], how='left', on=['user_id'])
    feature_data = pd.merge(feature_data, i_i[0],  how='left', on=['item_id'])
    feature_data = pd.merge(feature_data, c[0],  how='left', on=['item_category'])
    feature_data = pd.merge(feature_data, ic[0], how='left', on=['item_id','item_category'])
    feature_data = pd.merge(feature_data, ui[0], how='left', on=['user_id','item_id'])  ###different with example
    #feature_data = pd.merge(feature_data, uc[0], how='left', on=['user_id','item_category'])

    feature_data.fillna(-1, inplace=True)# 一个重要的位置###############
    
    #train_data_df_part_1 = pd.merge(train_data_df_part_1, df_part_1_UC, how='left', on=['user_id','item_category'])
    train_data = feature_data
    end_time = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    train_data.to_csv(train_data_filename,index=False)
    print('end time:', end_time)